In [31]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)


try:
    train = pd.read_csv('../data/train_clean.csv')
    test = pd.read_csv('../data/test_clean.csv')
    print("Datasets cargados exitosamente")
except FileNotFoundError:
    print("No se encontraron los archivos")
    exit()

print(f"✓ Dataset de entrenamiento: {train.shape}")
print(f"✓ Dataset de prueba: {test.shape}")

Datasets cargados exitosamente
✓ Dataset de entrenamiento: (7030723, 16)
✓ Dataset de prueba: (7027943, 16)


In [32]:
# DESCRIPCIÓN GENERAL DEL DATASET

print("\n" + "="*80)
print("1. DESCRIPCIÓN GENERAL DEL DATASET")
print("="*80)

def analizar_dataset(df, nombre):
    print(f"\n DATASET {nombre.upper()}")
    print("-" * 50)
    
    print(f"Dimensiones: {df.shape[0]:,} observaciones x {df.shape[1]} variables")
    
    print(f"\nTipos de datos:")
    tipos = df.dtypes.value_counts()
    for tipo, count in tipos.items():
        print(f" {tipo}: {count} variables")
    
    memoria_mb = df.memory_usage(deep=True).sum() / 1024**2
    print(f"\nMemoria utilizada: {memoria_mb:.2f} MB")

    print(f"\nDetalle de variables:")
    print(f"  Variables categóricas: user_id, merchant_id, age_range, gender, label")
    print(f"  Variables numéricas: activity_len, actions_*, unique_*, day_span, has_1111")
    print(f"  Variables temporales: date_min, date_max")
    
    return df.dtypes

print("\n Información detallada por dataset:")
tipos_train = analizar_dataset(train, "ENTRENAMIENTO")
tipos_test = analizar_dataset(test, "PRUEBA")

print(f"\n Comparación de estructuras:")
columnas_comunes = set(train.columns) & set(test.columns)
columnas_solo_train = set(train.columns) - set(test.columns)
columnas_solo_test = set(test.columns) - set(train.columns)

print(f" Columnas comunes: {len(columnas_comunes)}")
if columnas_solo_train:
    print(f" Solo en train: {list(columnas_solo_train)}")
if columnas_solo_test:
    print(f" Solo en test: {list(columnas_solo_test)}")


1. DESCRIPCIÓN GENERAL DEL DATASET

 Información detallada por dataset:

 DATASET ENTRENAMIENTO
--------------------------------------------------
Dimensiones: 7,030,723 observaciones x 16 variables

Tipos de datos:
 int64: 14 variables
 object: 2 variables

Memoria utilizada: 1649.24 MB

Detalle de variables:
  Variables categóricas: user_id, merchant_id, age_range, gender, label
  Variables numéricas: activity_len, actions_*, unique_*, day_span, has_1111
  Variables temporales: date_min, date_max

 DATASET PRUEBA
--------------------------------------------------
Dimensiones: 7,027,943 observaciones x 16 variables

Tipos de datos:
 int64: 13 variables
 object: 2 variables
 float64: 1 variables

Memoria utilizada: 1648.58 MB

Detalle de variables:
  Variables categóricas: user_id, merchant_id, age_range, gender, label
  Variables numéricas: activity_len, actions_*, unique_*, day_span, has_1111
  Variables temporales: date_min, date_max

 Comparación de estructuras:
 Columnas comunes:

In [33]:

# ANÁLISIS DE VARIABLES NUMÉRICAS

variables_numericas = [
    'activity_len', 'actions_0', 'actions_2', 'actions_3',
    'unique_items', 'unique_categories', 'unique_brands', 
    'day_span', 'has_1111'
]

variables_ordinales = ['age_range', 'gender']

def estadisticas_descriptivas(df, variables, nombre_dataset):
    print(f"\n ESTADÍSTICAS DESCRIPTIVAS - {nombre_dataset.upper()}")
    print("-" * 60)
    
    # Seleccionar solo variables que existen en el dataset
    vars_disponibles = [var for var in variables if var in df.columns]
    
    if not vars_disponibles:
        print("No se encontraron variables numéricas")
        return

    stats = df[vars_disponibles].describe()
    
    stats_extra = pd.DataFrame({
        'missing': df[vars_disponibles].isnull().sum(),
        'missing_pct': (df[vars_disponibles].isnull().sum() / len(df) * 100),
        'zeros': (df[vars_disponibles] == 0).sum(),
        'zeros_pct': ((df[vars_disponibles] == 0).sum() / len(df) * 100)
    })

    print("\n Medidas de tendencia central y dispersion:")
    print(stats.round(3))
    
    print(f"\n Valores faltantes y ceros:")
    print(stats_extra.round(2))
    
    print(f"\n Análisis detallado por variable:")
    for var in vars_disponibles:
        serie = df[var].dropna()
        if len(serie) == 0:
            continue
            
        print(f"\n {var.upper()}:")
        print(f"    - Rango: [{serie.min():.1f}, {serie.max():.1f}]")
        print(f"    - Media: {serie.mean():.3f} ± {serie.std():.3f}")
        print(f"    - Mediana: {serie.median():.1f}")
        print(f"    - Valores únicos: {serie.nunique():,}")
        
        # Percentiles
        p25, p75 = serie.quantile([0.25, 0.75])
        iqr = p75 - p25
        print(f"    - IQR: {iqr:.3f} (Q1={p25:.1f}, Q3={p75:.1f})")
        
        # Detección de outliers usando IQR
        limite_inf = p25 - 1.5 * iqr
        limite_sup = p75 + 1.5 * iqr
        outliers = ((serie < limite_inf) | (serie > limite_sup)).sum()
        outliers_pct = outliers / len(serie) * 100
        print(f"    - Outliers potenciales: {outliers:,} ({outliers_pct:.2f}%)")

# Análizar train
estadisticas_descriptivas(train, variables_numericas, "ENTRENAMIENTO")

# Analizar test (solo si tiene las mismas variables)
estadisticas_descriptivas(test, variables_numericas, "PRUEBA")


 ESTADÍSTICAS DESCRIPTIVAS - ENTRENAMIENTO
------------------------------------------------------------

 Medidas de tendencia central y dispersion:
       activity_len    actions_0    actions_2    actions_3  unique_items  unique_categories  unique_brands     day_span     has_1111
count   7030723.000  7030723.000  7030723.000  7030723.000   7030723.000        7030723.000    7030723.000  7030723.000  7030723.000
mean          3.499        3.447        0.233        0.215         2.267              1.347          1.046        7.373        0.198
std           5.550       11.546        0.727        0.932         4.900              1.103          0.312       25.831        0.399
min           0.000        0.000        0.000        0.000         0.000              0.000          0.000        0.000        0.000
25%           1.000        1.000        0.000        0.000         1.000              1.000          1.000        0.000        0.000
50%           1.000        1.000        0.000       

In [34]:

# ANÁLISIS DE VARIABLES CATEGÓRICAS

def analizar_categoricas(df, nombre_dataset):
    print(f"\n  VARIABLES CATEGÓRICAS - {nombre_dataset.upper()}")
    print("-" * 50)
    
    variables_cat = ['age_range', 'gender', 'label']
    
    for var in variables_cat:
        if var not in df.columns:
            continue
            
        print(f"\n  {var.upper()}:")
        
        conteos = df[var].value_counts(dropna=False)
        porcentajes = df[var].value_counts(normalize=True, dropna=False) * 100
        
        tabla = pd.DataFrame({
            'Conteo': conteos,
            'Porcentaje': porcentajes
        }).round(2)
        
        print(tabla)

        print(f"    - Valores únicos: {df[var].nunique()}")
        print(f"    - Valores nulos: {df[var].isnull().sum():,} ({df[var].isnull().sum()/len(df)*100:.2f}%)")

analizar_categoricas(train, "ENTRENAMIENTO")
analizar_categoricas(test, "PRUEBA")



  VARIABLES CATEGÓRICAS - ENTRENAMIENTO
--------------------------------------------------

  AGE_RANGE:
    Conteo  Porcentaje
3  1913722       27.22
4  1459923       20.76
0  1371222       19.50
5   752927       10.71
2   731938       10.41
6   655922        9.33
7   124493        1.77
8    20290        0.29
1      286        0.00
    - Valores únicos: 9
    - Valores nulos: 0 (0.00%)

  GENDER:
    Conteo  Porcentaje
0  5101730       72.56
1  1618110       23.01
2   310883        4.42
    - Valores únicos: 3
    - Valores nulos: 0 (0.00%)

  LABEL:
     Conteo  Porcentaje
-1  6769859       96.29
 0   244912        3.48
 1    15952        0.23
    - Valores únicos: 3
    - Valores nulos: 0 (0.00%)

  VARIABLES CATEGÓRICAS - PRUEBA
--------------------------------------------------

  AGE_RANGE:
    Conteo  Porcentaje
3  1916611       27.27
4  1460542       20.78
0  1364985       19.42
5   752608       10.71
2   733323       10.43
6   650358        9.25
7   128644        1.83
8    20

In [35]:
# ANÁLISIS DE VARIABLES TEMPORALES

def analizar_temporales(df, nombre_dataset):
    print(f"\n VARIABLES TEMPORALES - {nombre_dataset.upper()}")
    print("-" * 50)

    for col in ['date_min', 'date_max']:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_datetime(df[col], errors='coerce')
    
    if 'date_min' in df.columns:
        fecha_min = df['date_min'].dropna()
        print(f"\n  • DATE_MIN:")
        if len(fecha_min) > 0:
            print(f"    - Rango: {fecha_min.min()} a {fecha_min.max()}")
            print(f"    - Valores no nulos: {len(fecha_min):,} ({len(fecha_min)/len(df)*100:.2f}%)")
            
            meses = fecha_min.dt.month.value_counts().sort_index()
            print(f"    - Distribución por mes: {dict(meses)}")
        else:
            print("    - No hay fechas válidas")
    
    if 'date_max' in df.columns:
        fecha_max = df['date_max'].dropna()
        print(f"\n  • DATE_MAX:")
        if len(fecha_max) > 0:
            print(f"    - Rango: {fecha_max.min()} a {fecha_max.max()}")
            print(f"    - Valores no nulos: {len(fecha_max):,} ({len(fecha_max)/len(df)*100:.2f}%)")
        else:
            print("    - No hay fechas válidas")
    
    if 'day_span' in df.columns:
        span = df['day_span'].dropna()
        print(f"\n  • DAY_SPAN (días entre primera y última interacción):")
        if len(span) > 0:
            print(f"    - Media: {span.mean():.2f} días")
            print(f"    - Mediana: {span.median():.0f} días")
            print(f"    - Máximo: {span.max():.0f} días")
            print(f"    - Usuarios con span = 0: {(span == 0).sum():,} ({(span == 0).sum()/len(span)*100:.2f}%)")
        else:
            print("    - No hay datos de span disponibles")

analizar_temporales(train, "ENTRENAMIENTO")
analizar_temporales(test, "PRUEBA")



 VARIABLES TEMPORALES - ENTRENAMIENTO
--------------------------------------------------

  • DATE_MIN:
    - Rango: 2014-05-11 00:00:00 a 2014-11-12 00:00:00
    - Valores no nulos: 7,027,748 (99.96%)
    - Distribución por mes: {5: 473191, 6: 876483, 7: 691220, 8: 704240, 9: 910835, 10: 1245551, 11: 2126228}

  • DATE_MAX:
    - Rango: 2014-05-11 00:00:00 a 2014-11-12 00:00:00
    - Valores no nulos: 7,027,748 (99.96%)

  • DAY_SPAN (días entre primera y última interacción):
    - Media: 7.37 días
    - Mediana: 0 días
    - Máximo: 184 días
    - Usuarios con span = 0: 5,630,998 (80.09%)

 VARIABLES TEMPORALES - PRUEBA
--------------------------------------------------

  • DATE_MIN:
    - Rango: 2014-05-11 00:00:00 a 2014-11-12 00:00:00
    - Valores no nulos: 7,024,937 (99.96%)
    - Distribución por mes: {5: 475364, 6: 875377, 7: 693543, 8: 701605, 9: 910088, 10: 1244282, 11: 2124678}

  • DATE_MAX:
    - Rango: 2014-05-11 00:00:00 a 2014-11-12 00:00:00
    - Valores no nulos: 7

In [36]:
# ANÁLISIS ESPECÍFICO DEL COMPORTAMIENTO DE USUARIOS

def analizar_comportamiento(df, nombre_dataset):
    print(f"\n COMPORTAMIENTO DE USUARIOS - {nombre_dataset.upper()}")
    print("-" * 50)
    
    if 'activity_len' in df.columns:
        print(f"\n  • NIVEL DE ACTIVIDAD:")
        
        df_temp = df.copy()
        df_temp['actividad_nivel'] = pd.cut(
            df_temp['activity_len'], 
            bins=[0, 1, 3, 10, float('inf')], 
            labels=['Baja (1)', 'Media (2-3)', 'Alta (4-10)', 'Muy Alta (>10)'],
            include_lowest=True
        )
        
        actividad_dist = df_temp['actividad_nivel'].value_counts()
        print(f"    Distribución por nivel:")
        for nivel, count in actividad_dist.items():
            pct = count / len(df_temp) * 100
            print(f"    - {nivel}: {count:,} ({pct:.2f}%)")
    
    acciones_cols = ['actions_0', 'actions_2', 'actions_3']
    acciones_disponibles = [col for col in acciones_cols if col in df.columns]
    
    if acciones_disponibles:
        print(f"\n  • TIPOS DE ACCIONES:")
        acciones_totales = df[acciones_disponibles].sum()
        acciones_pct = acciones_totales / acciones_totales.sum() * 100
        
        nombres_acciones = {
            'actions_0': 'Vistas/Clics',
            'actions_2': 'Carrito',
            'actions_3': 'Compras'
        }
        
        for col in acciones_disponibles:
            nombre = nombres_acciones.get(col, col)
            print(f"    - {nombre}: {acciones_totales[col]:,} ({acciones_pct[col]:.2f}%)")
    
    diversidad_cols = ['unique_items', 'unique_categories', 'unique_brands']
    diversidad_disponibles = [col for col in diversidad_cols if col in df.columns]
    
    if diversidad_disponibles:
        print(f"\n  • DIVERSIDAD DE INTERACCIONES:")
        for col in diversidad_disponibles:
            media = df[col].mean()
            mediana = df[col].median()
            print(f"    - {col.replace('unique_', '').title()}: μ={media:.2f}, mediana={mediana:.0f}")

analizar_comportamiento(train, "ENTRENAMIENTO")
analizar_comportamiento(test, "PRUEBA")


 COMPORTAMIENTO DE USUARIOS - ENTRENAMIENTO
--------------------------------------------------

  • NIVEL DE ACTIVIDAD:
    Distribución por nivel:
    - Baja (1): 3,633,744 (51.68%)
    - Media (2-3): 1,725,981 (24.55%)
    - Alta (4-10): 1,188,907 (16.91%)
    - Muy Alta (>10): 482,091 (6.86%)

  • TIPOS DE ACCIONES:
    - Vistas/Clics: 24,235,247 (88.50%)
    - Carrito: 1,636,724 (5.98%)
    - Compras: 1,512,086 (5.52%)

  • DIVERSIDAD DE INTERACCIONES:
    - Items: μ=2.27, mediana=1
    - Categories: μ=1.35, mediana=1
    - Brands: μ=1.05, mediana=1

 COMPORTAMIENTO DE USUARIOS - PRUEBA
--------------------------------------------------

  • NIVEL DE ACTIVIDAD:
    Distribución por nivel:
    - Baja (1): 3,628,073 (51.62%)
    - Media (2-3): 1,729,124 (24.60%)
    - Alta (4-10): 1,188,581 (16.91%)
    - Muy Alta (>10): 482,165 (6.86%)

  • TIPOS DE ACCIONES:
    - Vistas/Clics: 24,315,466 (88.58%)
    - Carrito: 1,640,984 (5.98%)
    - Compras: 1,493,637 (5.44%)

  • DIVERSIDAD DE

In [37]:

# ANÁLISIS DE LA VARIABLE OBJETIVO (solo para train)

if 'label' in train.columns:
    print(f"\n VARIABLE OBJETIVO (LABEL) - ENTRENAMIENTO")
    print("-" * 50)
    
    label_counts = train['label'].value_counts(dropna=False)
    label_pcts = train['label'].value_counts(normalize=True, dropna=False) * 100
    
    print(f"\nDistribución completa:")
    for valor, count in label_counts.items():
        pct = label_pcts[valor]
        if valor == -1:
            descripcion = "(no son clientes nuevos - contexto)"
        elif valor == 0:
            descripcion = "(clientes nuevos, no recurrentes)"
        elif valor == 1:
            descripcion = "(clientes nuevos, recurrentes)"
        else:
            descripcion = ""
        
        print(f"  • Label {valor}: {count:,} ({pct:.2f}%) {descripcion}")
    
    clientes_nuevos = train[train['label'].isin([0, 1])]
    
    if len(clientes_nuevos) > 0:
        print(f"\nAnálisis para CLIENTES NUEVOS solamente:")
        print(f"  • Total clientes nuevos: {len(clientes_nuevos):,}")
        
        nuevos_counts = clientes_nuevos['label'].value_counts()
        nuevos_pcts = clientes_nuevos['label'].value_counts(normalize=True) * 100
        
        for valor, count in nuevos_counts.items():
            pct = nuevos_pcts[valor]
            tipo = "No recurrentes" if valor == 0 else "Recurrentes"
            print(f"  • {tipo}: {count:,} ({pct:.2f}%)")
        
        if 1 in nuevos_counts and 0 in nuevos_counts:
            ratio = nuevos_counts[0] / nuevos_counts[1]
            print(f"  • Ratio desbalance (0:1): {ratio:.2f}:1")
else:
    print("No se encontró la variable 'label' en el dataset de entrenamiento")




 VARIABLE OBJETIVO (LABEL) - ENTRENAMIENTO
--------------------------------------------------

Distribución completa:
  • Label -1: 6,769,859 (96.29%) (no son clientes nuevos - contexto)
  • Label 0: 244,912 (3.48%) (clientes nuevos, no recurrentes)
  • Label 1: 15,952 (0.23%) (clientes nuevos, recurrentes)

Análisis para CLIENTES NUEVOS solamente:
  • Total clientes nuevos: 260,864
  • No recurrentes: 244,912 (93.88%)
  • Recurrentes: 15,952 (6.12%)
  • Ratio desbalance (0:1): 15.35:1
